# 2. UNIFICACIÓN Y LIMPIEZA DE PUNTOS FIRMS-UNGRD #

- Concatenación de capas FIRMS de distintos sensores
- Normalización de rutas
- Eliminación de duplicados exactos priorizando MODIS

## 2.1 Unificación puntos de la plataforma FIRMS por año para Colombia y para el Santuario de Fauna y Flora de Iguaque ##

Se unifica múltiples shapefiles de puntos calientes (FIRMS) correspondientes por año (distintas fuentes/sensores/descargas) en una sola capa consolidada, preservando atributos y geometría. Obteniendo un shapefile tipo punto por año con el compilado de puntos dentro del AOI. Evitando conteos inflados por duplicidad.

### 2.1.1 Parte 1 ###

Carga de dependencias necesarias; definición de las rutas donde estan los shapefile y la ruta donde estara el shapefile el consolidado. Se lee cada shapefile en un geodataframe y verificación de CRS. Se une registros de varias capas en un único GeoDataFrame. Se normaliza la fecha. Se verifican campos clave y conversión a numero en caso de ser necesario. Se identifican los registros con la misma combinación de fechas, tanto para FIRMS Colombia como para FIRMS Santuario de Fauna y Flora de Iguaque. En caso de existir duplicados se debe preservar la información del instrumento MODIS. Se debe ajustar las rutas de acuerdo a los .shp por año disponibles para obtener el número de puntos consolidados por año

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
from src.firms_merge import setup_logger, merge_firms_shps

paths = [
    r"D:\Maestria_Geomatica\Semestre_IV\Puntos_calientes_Colombia_FIRMS_25_09_19\2024_DL_FIRE_J1V-C2_659709\fire_archive_J1V-C2_659709_wgs84_recorte.shp",
    r"D:\Maestria_Geomatica\Semestre_IV\Puntos_calientes_Colombia_FIRMS_25_09_19\2024_DL_FIRE_J2V-C2_659710\fire_nrt_J2V-C2_659710_wgs84_recorte.shp",
    r"D:\Maestria_Geomatica\Semestre_IV\Puntos_calientes_Colombia_FIRMS_25_09_19\2024_DL_FIRE_M-C61_659707\fire_archive_M-C61_659707_wgs84_recorte.shp",
    r"D:\Maestria_Geomatica\Semestre_IV\Puntos_calientes_Colombia_FIRMS_25_09_19\2024_DL_FIRE_SV-C2_659711\fire_archive_SV-C2_659711_wgs84_recorte.shp",
    r"D:\Maestria_Geomatica\Semestre_IV\Puntos_calientes_Iguaque_FIRMS_25_09_19\2024_DL_FIRE_J1V-C2_661783\fire_archive_J1V-C2_661783.shp",
    r"D:\Maestria_Geomatica\Semestre_IV\Puntos_calientes_Iguaque_FIRMS_25_09_19\2024_DL_FIRE_J2V-C2_661784\fire_nrt_J2V-C2_661784.shp",
    r"D:\Maestria_Geomatica\Semestre_IV\Puntos_calientes_Iguaque_FIRMS_25_09_19\2024_DL_FIRE_M-C61_661781\fire_archive_M-C61_661781.shp",
    r"D:\Maestria_Geomatica\Semestre_IV\Puntos_calientes_Iguaque_FIRMS_25_09_19\2024_DL_FIRE_SV-C2_661785\fire_archive_SV-C2_661785.shp",
]

out_shp = r"D:\Maestria_Geomatica\Semestre_IV\Puntos_unificados_25_11_05\puntos_unificados_2024_2.shp"
log_path = os.path.join(os.path.dirname(out_shp), "log_unificacion_2.txt")
logger = setup_logger(log_path)

final_path, summary = merge_firms_shps(
    paths=paths,
    out_path=out_shp,
    epsg_expected=4326,
    prefer_instrument="MODIS",
    version_out=True,
    logger=logger
)

print("✅ Salida:", final_path)
print(summary)
logger.info(f"RESUMEN: {summary}")

C:\Users\Equipo\Tesis\SemestreIV\Objetivo1\Objetivo1_26_02_20\src\firms_merge.py:61: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, dayfirst=True, errors="raise").date().isoformat()


✅ Salida: D:\Maestria_Geomatica\Semestre_IV\Puntos_unificados_25_11_05\puntos_unificados_2024_2.shp
MergeSummary(n_inputs=8, n_features_in=2902, n_features_out=1834, n_dropped_dupes=1068, crs_out=<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
)


C:\Users\Equipo\Tesis\SemestreIV\Objetivo1\Objetivo1_26_02_20\src\firms_merge.py:155: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(final_path, driver="ESRI Shapefile", encoding="utf-8")
C:\Users\Equipo\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field ACQ_DATE create as date field, though DateTime requested.
  ogr_write(
C:\Users\Equipo\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'ACQ_DATE_NORM' to 'ACQ_DATE_N'
  ogr_write(


## 2.2 Unificación de Todos los Puntos Calientes Reportados Por la Plataforma FIRMS para los Años de Estudio ##

Se unifican múltiples shapefiles ubicados en una carpeta, produciendo un único shapefile consolidado


### 2.2.1 Parte 1 ###

Se cargan las librerias. Se define carpeta y patrón de búsqueda. Se lee cada shapefile y se define el CRS, normalizando nombres de columnas. Finalmente se unen todos geodataframe en uno solo. Se define el nombre del archivo y la ruta de salida. Se resume las operaciones realizadas

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import os
from src.shp_merge_folder import setup_logger, merge_shapefiles_in_folder, format_report

carpeta_shp = r"D:\Maestria_Geomatica\Semestre_IV\Puntos_unificados_25_11_05\Puntos_unificados_26_02_23" 
out_name = "puntos_unificados_FIRMS_26_02_23.shp"

log_path = os.path.join(carpeta_shp, "log_merge_folder.txt")
logger = setup_logger(log_path)

summary = merge_shapefiles_in_folder(
    folder=carpeta_shp,
    out_name=out_name,
    pattern="*.shp",
    normalize_cols=True,
    reproject_to_base=True,
    version_out=True,      # ✅ versionar (NO borrar/sobrescribir)
    logger=logger
)

print(format_report(summary))


--- REPORTE FINAL ---
📁 Carpeta: D:\Maestria_Geomatica\Semestre_IV\Puntos_unificados_25_11_05\Puntos_unificados_26_02_23
🔎 Shapefiles encontrados: 11
✅ Leídos y usados: 11
⚠️ Omitidos (vacíos): 0
❌ Con error: 0
📌 Registros salida: 6744
🧭 CRS final: EPSG:4326
🧩 Tipos geometría: ['Point']
🧾 Columnas: ['latitude', 'longitude', 'brightness', 'scan', 'track', 'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence', 'version', 'bright_t31', 'frp', 'daynight', 'type', 'acq_date_n', 'geometry']
💾 Salida: D:\Maestria_Geomatica\Semestre_IV\Puntos_unificados_25_11_05\Puntos_unificados_26_02_23\puntos_unificados_FIRMS_26_02_23.shp



C:\Users\Equipo\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field acq_date create as date field, though DateTime requested.
  ogr_write(
